Notebook for running models

In [ ]:
from utils import visualization

from models/3D_CNN import model
from models/3D_CNN import train
from models/3D_CNN import evaluate

CNN_model = model()
train(CNN_model)
results = evaluate(model)

visualization(results)